# Analyze results for fibroblast states

In [ ]:
import numpy as np
import pandas as pd
import anndata
import os
import scipy
import h5py
from grelu.interpret.motifs import trim_pwm
from grelu.visualize import plot_attributions
from plotnine import *
from grelu.io.motifs import read_meme_file
from tangermeme.plot import plot_pwm
%matplotlib inline

## Paths

In [ ]:
save_dir="/gstore/data/resbioai/grelu/decima/20240823"
matrix_file = os.path.join(save_dir, "data.h5ad")

## Load data

In [ ]:
ad = anndata.read_h5ad(matrix_file)
ad = ad[ad.obs.cell_type=='fibroblast']

In [ ]:
motifs = read_meme_file('/gstore/data/resbioai/grelu/decima/H12CORE_meme_format.meme')

## Predict differential expression

In [ ]:
ad.var['diff_true'] = ad[ad.obs.organ == 'heart'].X.mean(0) - ad[ad.obs.organ != 'heart'].X.mean(0)
ad.var['diff_pred'] =  ad[ad.obs.organ == 'heart'].layers['preds'].mean(0) - ad[ad.obs.organ != 'heart'].layers['preds'].mean(0)

In [ ]:
print(scipy.stats.pearsonr(ad.var.loc[ad.var.dataset=='test', 'diff_true'], ad.var.loc[ad.var.dataset=='test', 'diff_pred']))

In [ ]:
(
    ggplot(ad.var[ad.var.dataset=='test'], aes(x='diff_true', y='diff_pred')) 
    + geom_pointdensity(size=.1) + theme_classic() + theme(figure_size=(2.6, 2.5))
    + xlab('Measured log FC') + ylab('Predicted logFC')
    + ggtitle('           Fibroblast\n(cardiac vs. noncardiac)')
    + geom_abline(slope=1, intercept=0)
    + geom_vline(xintercept = 0, linetype='--')
    + geom_hline(yintercept = 0, linetype='--')
)

In [ ]:
plot_pwm(motifs['GATA4.H12CORE.0.PSM.A'])